## Milestone 1 
### Preparing the Energy and Temperture Data

In [2]:
#Libraries
import pandas as pd


In [3]:
#importing the data into dataframes
#temperture data
hr_temp = pd.read_csv("hr_temp_20170201-20200131_subset.csv")
#energy data
hrl_metered = pd.read_csv("hrl_load_metered - 20170201-20200131.csv")


In [4]:
#date is done hourly; checking for any nulls in the data
#hr_temp.head()
hr_temp.isnull().sum()

STATION                      0
DATE                         0
REPORT_TYPE                  0
SOURCE                       0
HourlyDryBulbTemperature    37
dtype: int64

In [5]:
hr_temp['DATE'] = pd.to_datetime(hr_temp['DATE'],format='%Y-%m-%d %H:%M:%S')
hr_temp.dtypes

STATION                              int64
DATE                        datetime64[ns]
REPORT_TYPE                         object
SOURCE                               int64
HourlyDryBulbTemperature           float64
dtype: object

In [6]:
hr_temp.head()

,STATION,DATE,REPORT_TYPE,SOURCE,HourlyDryBulbTemperature
0,72520514762,2017-02-01 00:53:00,FM-15,7,37.0
1,72520514762,2017-02-01 01:53:00,FM-15,7,37.0
2,72520514762,2017-02-01 02:53:00,FM-15,7,36.0
3,72520514762,2017-02-01 03:53:00,FM-15,7,36.0
4,72520514762,2017-02-01 04:53:00,FM-15,7,36.0


In [7]:
#interpolation for missing values
hr_temp['HourlyDryBulbTemperature'].interpolate(inplace=True)

#Extract all the date time information for joins & et
hr_temp['hour'] = pd.DatetimeIndex(hr_temp['DATE']).hour
hr_temp['weekday'] = pd.DatetimeIndex(hr_temp['DATE']).weekday
hr_temp['month'] = pd.DatetimeIndex(hr_temp['DATE']).month
hr_temp['year'] = pd.DatetimeIndex(hr_temp['DATE']).year
hr_temp['date4join'] = pd.to_datetime(hr_temp['DATE']).dt.date

In [8]:
hr_temp.isnull().sum()

STATION                     0
DATE                        0
REPORT_TYPE                 0
SOURCE                      0
HourlyDryBulbTemperature    0
hour                        0
weekday                     0
month                       0
year                        0
date4join                   0
dtype: int64

In [9]:
#date is done hourly; checking for any nulls in the data

hrl_metered.head()

hrl_metered.isnull().sum()

datetime_beginning_utc    0
datetime_beginning_ept    0
nerc_region               0
mkt_region                0
zone                      0
load_area                 0
mw                        0
is_verified               0
dtype: int64

In [10]:
#use datetime_beginning_ept for date
hrl_metered['DATE'] = pd.to_datetime(hrl_metered['datetime_beginning_ept'],format='%m/%d/%Y %H:%M')
hrl_metered['date4join'] = pd.to_datetime(hrl_metered['DATE']).dt.date
hrl_metered['hour'] = pd.DatetimeIndex(hrl_metered['DATE']).hour
hrl_metered.head()

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified,DATE,date4join,hour
0,2/1/2017 5:00,2/1/2017 0:00,RFC,WEST,DUQ,DUQ,1419.881,True,2017-02-01 00:00:00,2017-02-01,0
1,2/1/2017 6:00,2/1/2017 1:00,RFC,WEST,DUQ,DUQ,1379.505,True,2017-02-01 01:00:00,2017-02-01,1
2,2/1/2017 7:00,2/1/2017 2:00,RFC,WEST,DUQ,DUQ,1366.106,True,2017-02-01 02:00:00,2017-02-01,2
3,2/1/2017 8:00,2/1/2017 3:00,RFC,WEST,DUQ,DUQ,1364.453,True,2017-02-01 03:00:00,2017-02-01,3
4,2/1/2017 9:00,2/1/2017 4:00,RFC,WEST,DUQ,DUQ,1391.265,True,2017-02-01 04:00:00,2017-02-01,4


In [11]:
energy_data = hrl_metered[['date4join','hour','mw']]

In [12]:
#Create dataframe for the combined energy and temperture data 
temp_energy = hr_temp.merge(energy_data,how='left',on=['date4join','hour'])
temp_energy.rename(columns={'DATE':'date','HourlyDryBulbTemperature':'temp'},inplace=True)
temp_energy.head()

,STATION,date,REPORT_TYPE,SOURCE,temp,hour,weekday,month,year,date4join,mw
0,72520514762,2017-02-01 00:53:00,FM-15,7,37.0,0,2,2,2017,2017-02-01,1419.881
1,72520514762,2017-02-01 01:53:00,FM-15,7,37.0,1,2,2,2017,2017-02-01,1379.505
2,72520514762,2017-02-01 02:53:00,FM-15,7,36.0,2,2,2,2017,2017-02-01,1366.106
3,72520514762,2017-02-01 03:53:00,FM-15,7,36.0,3,2,2,2017,2017-02-01,1364.453
4,72520514762,2017-02-01 04:53:00,FM-15,7,36.0,4,2,2,2017,2017-02-01,1391.265


In [13]:
#dropping extra columns and setting the date as the index
temp_energy.drop(columns=['STATION','REPORT_TYPE','SOURCE','date4join'],inplace=True)
temp_energy = temp_energy[['date','mw','temp','hour','weekday','month','year']]
temp_energy.set_index('date',inplace=True)
temp_energy.head()

,mw,temp,hour,weekday,month,year
date,,,,,,
2017-02-01 00:53:00,1419.881,37.0,0,2,2,2017
2017-02-01 01:53:00,1379.505,37.0,1,2,2,2017
2017-02-01 02:53:00,1366.106,36.0,2,2,2,2017
2017-02-01 03:53:00,1364.453,36.0,3,2,2,2017
2017-02-01 04:53:00,1391.265,36.0,4,2,2,2017


In [15]:
"""
Subsetting the data into test & train based on the following:
    - Use all of the data up through 12-31-2019 for training
    - January 2020 as your testing datasets.

Note to self:
test = temp_energy.loc[temp_energy['date'] >= '2020-01-01']  - if date is not indexed
Good references for task 4:
https://statisticsglobe.com/set-datetimeindex-pandas-dataframe-python
https://stackoverflow.com/questions/44547401/pandas-selecting-dataframe-rows-between-two-dates-datetime-index
"""

training = temp_energy[(temp_energy.index.get_level_values(0) < '2020-01-01 00:00:00')]

training.head()

,mw,temp,hour,weekday,month,year
date,,,,,,
2017-02-01 00:53:00,1419.881,37.0,0,2,2,2017
2017-02-01 01:53:00,1379.505,37.0,1,2,2,2017
2017-02-01 02:53:00,1366.106,36.0,2,2,2,2017
2017-02-01 03:53:00,1364.453,36.0,3,2,2,2017
2017-02-01 04:53:00,1391.265,36.0,4,2,2,2017


In [14]:


test = temp_energy[(temp_energy.index.get_level_values(0) >= '2020-01-01 00:00:00')]
test.head()

,mw,temp,hour,weekday,month,year
date,,,,,,
2020-01-01 00:53:00,1363.428,31.0,0,2,1,2020
2020-01-01 01:53:00,1335.975,29.0,1,2,1,2020
2020-01-01 02:53:00,1296.817,30.0,2,2,1,2020
2020-01-01 03:53:00,1288.403,30.0,3,2,1,2020
2020-01-01 04:53:00,1292.263,31.0,4,2,1,2020
